<a href="https://colab.research.google.com/github/iceman011/mydeeplearning/blob/master/ez-mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import time


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt


class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5,lr =0.001, train_on_gpu=False,output_classes):
        ''' Builds a feedforward network with arbitrary hidden layers.
        
            Arguments
            ---------
            input_size: integer, size of the input layer
            output_size: integer, size of the output layer
            hidden_layers: list of integers, the sizes of the hidden layers
        
        '''
        super().__init__()
        if train_on_gpu:
          # check if CUDA is available
          self.train_on_gpu = torch.cuda.is_available()

        if not self.train_on_gpu:
            print('CUDA is not available.  Training on CPU ...')
        else:
            print('CUDA is available!  Training on GPU ...')
            
        self.input_size = input_size
        self.output_size = output_size
        self.learning_rate = lr
        self.output_classes = output_classes

        # Input to a hidden layer
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # Add a variable number of more hidden layers
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        for each in self.hidden_layers:
            x = F.relu(each(x))
            x = self.dropout(x)
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)


def validation(model, validationloader, criterion):
    accuracy = 0
    validation_loss = 0

    with torch.no_grad():    
      # move tensors to GPU if CUDA is available
      if model.train_on_gpu:
        model.cuda()

      for images, labels in validationloader:

          if model.train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
          
          images = images.resize_(images.size()[0], model.input_size)

          output = model.forward(images)
          validation_loss += criterion(output, labels).item()

          ## Calculating the accuracy 
          # Model's output is log-softmax, take exponential to get the probabilities
          ps = torch.exp(output)
          # Class with highest probability is our predicted class, compare with true label
          #equality = (labels.data == ps.max(1)[1])
          # Accuracy is number of correct predictions divided by all predictions, just take the mean
          #accuracy += equality.type_as(torch.FloatTensor()).mean()


        
          top_p, top_class = ps.topk(1, dim=1)
          equals = top_class == labels.view(*top_class.shape)
          accuracy += torch.mean(equals.type(torch.FloatTensor))      
        
      validation_loss = validation_loss/len(validationloader)
      accuracy = accuracy/len(validationloader)

    return validation_loss, accuracy

#############################
# TEST MODEL
############################
def test(model,test_loader,criterion,checkpoint):
    # track test loss
    test_loss = 0.0
    class_correct = list(0. for i in range(model.output_size))
    class_total = list(0. for i in range(model.output_size))

    with torch.no_grad():    
      model.eval()
      # iterate over test data
      for data, target in test_loader:
          # move tensors to GPU if CUDA is available
          if train_on_gpu:
              data, target = data.cuda(), target.cuda()
          # forward pass: compute predicted outputs by passing inputs to the model
          output = model(data)
          # calculate the batch loss
          loss = criterion(output, target)
          # update test loss 
          test_loss += loss.item()*data.size(0)
          # convert output probabilities to predicted class
          _, pred = torch.max(output, 1)    
          # compare predictions to true label
          correct_tensor = pred.eq(target.data.view_as(pred))
          correct = np.squeeze(correct_tensor.numpy()) if not model.train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
          # calculate test accuracy for each object class
          for i in len(target) # range(batch_size):
              label = target.data[i]
              class_correct[label] += correct[i].item()
              class_total[label] += 1

      # average test loss
      test_loss = test_loss/len(test_loader.dataset)
      print('Test Loss: {:.6f}\n'.format(test_loss))

      for i in range(model.output_size):
          if class_total[i] > 0:
            current_key.format('Test Accuracy of {%5s}',model.classes[i])
            current_val.format('{%2d%}% ({%2d}/{%2d})',100 * class_correct[i] / class_total[i],
                  np.sum(class_correct[i]), np.sum(class_total[i]))
            ele={current_key:current_vale}
            checkpoint.update(ele)

            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                  classes[i], 100 * class_correct[i] / class_total[i],
                  np.sum(class_correct[i]), np.sum(class_total[i])))
          else:
              print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

      current_key.format('Test Accuracy (Overall): ',100. * np.sum(class_correct) / np.sum(class_total))
      current_val.format('{%2d%}% ({%2d}/{%2d})', 100. * np.sum(class_correct) / np.sum(class_total),
          np.sum(class_correct), np.sum(class_total) )
      ele={current_key:current_vale}
      checkpoint.update(ele)
      
      print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
          100. * np.sum(class_correct) / np.sum(class_total),
          np.sum(class_correct), np.sum(class_total)))
    
    #model.train()
  
  return checkpoint


def train(model, trainloader, validationloader, criterion, optimizer, epochs=5, print_every=40 ):
    # monitor training loss    
    steps = 0
    start_time = time.time()
    train_losses, valid_losses = [], []
    
    # move tensors to GPU if CUDA is available
    if model.train_on_gpu:
      model.cuda()

    print('Starting Training using Model Parameters ',model )
    valid_loss_min = np.Inf # set initial "min" to infinity
    
    for e in range(epochs):        
        train_loss = 0.0
        valid_loss = 0.0


        ###################
        # train the model #
        ###################
        model.train() # prep model for training

        for images, labels in trainloader:
            steps += 1
            
            # move tensors to GPU if CUDA is available
            if model.train_on_gpu:
              images, labels = images.cuda(), labels.cuda()

            # Flatten images into a 784 long vector
            images.resize_(images.size()[0], 784)
            
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

        train_loss = train_loss/len(trainloader.sampler)
        train_losses.append(train_loss)
        ######################    
        # validate the model #
        ######################
        # Model in inference mode, dropout is off
        model.eval()
        
        # Turn off gradients for validation, will speed up inference
        with torch.no_grad():
            valid_loss, accuracy = validation(model, validationloader, criterion)
        
        valid_losses.append(valid_loss)

        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy : {:.6f}'.format(
            e+1, 
            train_loss,
            valid_loss,
            accuracy
            ))
        
        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f} ,)  Accuracy: {:.6f}  TimeElapsed: {:.6f}.  Saving model ...'.format(
            valid_loss_min,
            valid_loss,
            accuracy,
            (time.time() - start_time)))
              
            checkpoint = {'InputSize': model.input_size,
                  'OutputSize': model.output_size,
                  'HiddenLayers': [each.out_features for each in model.hidden_layers],
                  'LearningRate':model.learning_rate,
                  'TrainingLoss' :train_loss,
                  'ValidationLoss ':valid_loss,
                  'TrainingLosses' :train_losses,
                  'ValidationLosses ':valid_losses,
                  'ElapsedTime': (time.time() - start_time),
                  'CheckPointTimestamp': time.time(),
                  'CurrentEpoch': e,
                  'GPUState': model.train_on_gpu,
                  'OutputClasses',model.output_classes,
                  'StateDictionay': model.state_dict()}
            
            #print(checkpoint)
            torch.save(checkpoint, './'+str(start_time)+'/checkpoint_'+e+'.pt')
            valid_loss_min = valid_loss
    return train_losses , valid_losses ,checkpoint


def plotLossTrend(train_losses,validation_losses):
  plt.plot(train_losses, label='Training loss')
  plt.plot(validation_losses, label='Validation loss')
  plt.legend(frameon=False)



def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    print(' Loading Checkpoint from file :{} with TrainingLoss :{} , ValidationLoss :{} , ElapsedTime :{} , CheckPointTimeStamp :{} , LastEpoch :{} , GPUStatus :{}',
          filepath,checkpoint['TrainingLoss'],checkpoint['ValidatoinLoss'],checkpoint['ElapsedTime'],checkpoint['CheckPointTimestamp'],checkpoint['CurrentEpoch'],checkpoint['GPUStatus'])
    
    model = Network(checkpoint['InputSize'],
                             checkpoint['OutputSize'],
                             checkpoint['HiddenLayers'],
                            checkpoint['LearningRate'],
                            checkpoint['GPUState'],
                            checkpoint['OutputClasses']
                    )
    model.load_state_dict(checkpoint['state_dict'])
    return model , checkpoint

SyntaxError: ignored

In [0]:
        ######################    
        # LOADING DATA #
        ######################

# import libraries
import torch
import numpy as np

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [7]:
from torch import optim

        ######################    
        # MAIN #
        ######################

# specify the image classes
classes = ['1', '2', '3', '4', '5',
           '6', '7', '8', '9', '10']

# Create the network, define the criterion and optimizer

model = Network(784, 10, [512, 256, 128], lr=0.001,train_on_gpu=True, output_classes=classes)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_losses , valid_losses = train(model, train_loader, valid_loader, criterion, optimizer, epochs=10)

plotLossTrend (train_losses , valid_losses)

TypeError: ignored